In [1]:
!git clone https://github.com/Mykrass/sql-pandahouse-fbprophet.git

Cloning into 'sql-pandahouse-fbprophet'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 34 (delta 6), reused 6 (delta 0), pack-reused 0
Unpacking objects: 100% (34/34), done.


In [2]:
!ls -la ./sql-pandahouse-fbprophet

total 28
drwxr-xr-x 4 root root 4096 Oct  3 16:20 .
drwxr-xr-x 1 root root 4096 Oct  3 16:20 ..
drwxr-xr-x 8 root root 4096 Oct  3 16:20 .git
drwxr-xr-x 2 root root 4096 Oct  3 16:20 .idea
-rw-r--r-- 1 root root  102 Oct  3 16:20 README.md
-rw-r--r-- 1 root root   44 Oct  3 16:20 requirements.txt
-rw-r--r-- 1 root root  819 Oct  3 16:20 sql_query.py


In [3]:
!cat ./sql-pandahouse-fbprophet/requirements.txt

pandahouse >= 0.2.7 
pystan~=2.14
fbprophet


In [ ]:
!pip install -r ./sql-pandahouse-fbprophet/requirements.txt

In [5]:
%load_ext autoreload
%autoreload 2

#!python3 ./sql-pandahouse-fbprophet/sql_query.py --help
!cat ./sql-pandahouse-fbprophet/sql_query.py

%run ./sql-pandahouse-fbprophet/sql_query.py

import pandas as pd
import numpy as np
import pandahouse as ph
from datetime import date
import holidays
from fbprophet import Prophet


# Input parameter for CSV filename on Amazon S3
connection = dict(database='default',
                  host='http://clickhouse.beslan.pro:8080',
                  user='student',
                  password='dpo_python_2020')


#
def download(connection):
    query = """
    SELECT CAST(BuyDate AS Date) AS SalesDate, UserID, DeviceID, Rub FROM checks  AS l
    LEFT JOIN (SELECT * FROM devices) AS r ON l.UserID=r.UserID
    LIMIT 10000
    """
    df = ph.read_clickhouse(query, connection=connection)
    return df


#
def agregate(df):
    agg = df.groupby(['SalesDate', 'UserID'], as_index=False).sum()
    return agg

#
df = download(connection)
df_agg = agregate(df)
df_agg


In [6]:
df

,SalesDate,UserID,DeviceID,Rub
0,2019-10-04,18446583642950580515,17250723755157568386,3
1,2019-10-04,18446535622689003675,9561950678155309475,4
2,2019-10-04,18446130411954852964,5503112685478315026,0
3,2019-10-04,18446003252714243011,288146858669189183,3
4,2019-10-04,18445948434655311802,16879352180268202912,8
...,...,...,...,...
9995,2019-10-04,17405564575727614293,13893802921221960084,0
9996,2019-10-04,17405336384875763172,7733679835520304887,3
9997,2019-10-04,17404942641122915630,7735890600530526716,22
9998,2019-10-04,17404910798825845245,1286139160877468037,7


In [7]:
df_agg

,SalesDate,UserID,DeviceID,Rub
0,2019-10-04,17404868325863270232,13869044896282806272,2
1,2019-10-04,17404910798825845245,1286139160877468160,7
2,2019-10-04,17404942641122915630,7735890600530526208,22
3,2019-10-04,17405336384875763172,7733679835520305152,3
4,2019-10-04,17405564575727614293,13893802921221959680,0
...,...,...,...,...
9995,2019-10-04,18445948434655311802,16879352180268201984,8
9996,2019-10-04,18446003252714243011,288146858669189184,3
9997,2019-10-04,18446130411954852964,5503112685478315008,0
9998,2019-10-04,18446535622689003675,9561950678155309056,4


THE END